In [1]:
import os

In [2]:
%pwd

'd:\\UAE_USED_CAR_DEEP_LEARNING\\UAE_USED_CAR_DEEP_LEARNING_ANALYSIS_PROJECT\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\UAE_USED_CAR_DEEP_LEARNING\\UAE_USED_CAR_DEEP_LEARNING_ANALYSIS_PROJECT'

In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import get_file

In [6]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path   
    params_image_size : list
    params_learning_rate : float
    params_include_top : bool
    params_weights : str
    params_classes : int

In [7]:
from USED_CAR_PREDICTION.constants import *
from USED_CAR_PREDICTION.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):  
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
        
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path= Path(config.base_model_path),
            updated_base_model_path= Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes = self.params.CLASSES,
        )
        
        return prepare_base_model_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [10]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape= self.config.params_image_size,
            weights=self.config.params_weights,
            include_top = self.config.params_include_top
        )
        
        self.save_model(path = self.config.base_model_path, model=self.model)
        
        
        
    @staticmethod
    def _prepare_full_model(model, classes, freezer_all, freezer_till, learning_rate):
        if freezer_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freezer_till is not None) and (freezer_till > 0):
            for layer in model.layers[:-freezer_till]:
                layer.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation = 'softmax',
        )(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction,
        )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy']
        )
        
        
        full_model.summary()
        return full_model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freezer_all=True,
            freezer_till=None,
            learning_rate = self.config.params_learning_rate
            
        )
            
            
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [11]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-06-15 01:32:28,769 : INFO : YAML file config\config.yaml loaded successfully.]
[2025-06-15 01:32:28,773 : INFO : YAML file params.yaml loaded successfully.]
[2025-06-15 01:32:28,776 : INFO : created directory at: artifacts]
[2025-06-15 01:32:28,778 : INFO : created directory at: artifacts/prepare_base_model]
[2025-06-15 01:32:29,320 : WARNING : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                  

In [12]:
# from tensorflow.keras.applications import VGG16

# # Load the VGG16 model with pretrained ImageNet weights
# model = VGG16(weights='imagenet')

# # Print the model summary
# model.summary()
